In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import one_hot
import matplotlib.pyplot as py
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

In [4]:
df=pd.read_csv("Mental Health Dataset.csv")
df.posts=df.posts.astype(str)

In [5]:
df.isnull().sum()
print(df.tail())

                                                   posts      predicted  \
10387  hey everyone I am a 25 year old male I work ou...       negative   
10388  have surgery for stage 1 colon cancer 1 year a...  very negative   
10389  the doctor advise we he could not remove the a...        neutral   
10390  my 66 year old father have been through so muc...        neutral   
10391  I have bein have a bloody stool since last yea...       negative   

       intensity  
10387         -1  
10388         -2  
10389          0  
10390          0  
10391         -1  


In [6]:
def clean_text(text):
    text=text.lower()
    text=re.sub(r'[^a-zA-Z0-9\s]','',text)
    return text

In [7]:
df['posts']=df['posts'].apply(clean_text)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assuming df.posts is your text data
vocab_size=500
max_length = 200
embed_vector_size=200
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(df.posts)

# Convert text to sequences
encoded_reviews = tokenizer.texts_to_sequences(df.posts)

# Pad sequences
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')


In [9]:
#ANN BASED MODEL IS HERE
from tensorflow.keras.utils import to_categorical

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_reviews, df['intensity'], test_size=0.2, random_state=42)

# Build the model with Dropout layers
model = Sequential()
model.add(Embedding(vocab_size, embed_vector_size, input_length=max_length, name="embedding"))
model.add(Flatten())
model.add(Dropout(0.5))  # Adding dropout with a dropout rate of 0.5
model.add(Dense(4, activation='softmax'))  # Assuming you have 4 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the oversampled data
model.fit(X_train, to_categorical(y_train,num_classes=4), epochs=5, validation_data=(X_test, to_categorical(y_test, num_classes=4)))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, to_categorical(y_test, num_classes=4))
print(f"Accuracy on Test Set: {accuracy}")


Epoch 1/5
260/260 [==============================] - 13s 39ms/step - loss: 0.9201 - accuracy: 0.6200 - val_loss: 0.7941 - val_accuracy: 0.6720
Epoch 2/5
260/260 [==============================] - 3s 10ms/step - loss: 0.5993 - accuracy: 0.7702 - val_loss: 0.7475 - val_accuracy: 0.6965
Epoch 3/5
260/260 [==============================] - 2s 8ms/step - loss: 0.3824 - accuracy: 0.8818 - val_loss: 0.7725 - val_accuracy: 0.6917
Epoch 4/5
260/260 [==============================] - 1s 5ms/step - loss: 0.2382 - accuracy: 0.9364 - val_loss: 0.8379 - val_accuracy: 0.6797
Epoch 5/5
65/65 [==============================] - 0s 4ms/step - loss: 0.8689 - accuracy: 0.6768
Accuracy on Test Set: 0.6767676472663879


In [10]:
#DOUBLE LAYERED LSTM 

from tensorflow.keras.layers import LSTM

# Build the model with LSTM and Dropout layers
model = Sequential()
model.add(Embedding(vocab_size, embed_vector_size, input_length=max_length, name="embedding"))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(32))
model.add(Dropout(0.2))  # Adding dropout with a dropout rate of 0.2
model.add(Dense(4, activation='softmax'))  # Assuming you have 4 classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Use categorical_crossentropy for classification

# Train the model on the oversampled data
model.fit(X_train, to_categorical(y_train, num_classes=4), epochs=15, validation_data=(X_test, to_categorical(y_test, num_classes=4)))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, to_categorical(y_test, num_classes=4))
print(f"Accuracy on Test Set: {accuracy}")


Epoch 1/15
260/260 [==============================] - 15s 39ms/step - loss: 1.1118 - accuracy: 0.5254 - val_loss: 1.0045 - val_accuracy: 0.6123
Epoch 2/15
260/260 [==============================] - 6s 22ms/step - loss: 1.0912 - accuracy: 0.4992 - val_loss: 1.0932 - val_accuracy: 0.4598
Epoch 3/15
260/260 [==============================] - 5s 18ms/step - loss: 1.0079 - accuracy: 0.5809 - val_loss: 0.8843 - val_accuracy: 0.6494
Epoch 4/15
260/260 [==============================] - 6s 22ms/step - loss: 0.8847 - accuracy: 0.6542 - val_loss: 0.8469 - val_accuracy: 0.6522
Epoch 5/15
260/260 [==============================] - 4s 17ms/step - loss: 0.8212 - accuracy: 0.6751 - val_loss: 0.7992 - val_accuracy: 0.6888
Epoch 6/15
260/260 [==============================] - 4s 17ms/step - loss: 0.7563 - accuracy: 0.7058 - val_loss: 0.7528 - val_accuracy: 0.6931
Epoch 7/15
260/260 [==============================] - 6s 21ms/step - loss: 0.7399 - accuracy: 0.7131 - val_loss: 0.7206 - val_accuracy: 0.700

In [23]:
#GRU BASED MODEL
from tensorflow.keras.layers import Embedding, GRU
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length))
model.add(GRU(units=64, return_sequences=True))
model.add(GRU(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the oversampled data
model.fit(X_train, to_categorical(y_train,num_classes=4), epochs=15, validation_data=(X_test, to_categorical(y_test, num_classes=4)))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, to_categorical(y_test, num_classes=4))
print(f"Accuracy on Test Set: {accuracy}")


Epoch 1/15
260/260 [==============================] - 15s 45ms/step - loss: 1.1295 - accuracy: 0.4896 - val_loss: 0.9466 - val_accuracy: 0.6219
Epoch 2/15
260/260 [==============================] - 5s 17ms/step - loss: 0.9124 - accuracy: 0.6427 - val_loss: 0.8370 - val_accuracy: 0.6652
Epoch 3/15
260/260 [==============================] - 6s 22ms/step - loss: 0.8356 - accuracy: 0.6738 - val_loss: 0.8008 - val_accuracy: 0.6734
Epoch 4/15
260/260 [==============================] - 6s 22ms/step - loss: 0.7830 - accuracy: 0.6951 - val_loss: 0.8152 - val_accuracy: 0.6537
Epoch 5/15
260/260 [==============================] - 5s 19ms/step - loss: 0.7538 - accuracy: 0.7119 - val_loss: 0.7658 - val_accuracy: 0.7003
Epoch 6/15
260/260 [==============================] - 5s 19ms/step - loss: 0.7325 - accuracy: 0.7263 - val_loss: 0.7631 - val_accuracy: 0.7018
Epoch 7/15
260/260 [==============================] - 5s 18ms/step - loss: 0.7165 - accuracy: 0.7329 - val_loss: 0.7431 - val_accuracy: 0.700